In [5]:
import numpy as np

from keras import metrics

from sklearn.model_selection import StratifiedGroupKFold

from load_data import load_data 
from preprocess_data import preprocess_data 
from cnn import cnn

#y = x의 label List
#s = x의 subject List

#x = time x chnnel의 eeg signal(2d ndarray)를 요소로 갖는 List
x_ADHD, y_ADHD, s_ADHD = load_data("/home/unixuser/cdproject/public_data/ADHD", 1)
x_HC, y_HC, s_HC = load_data("/home/unixuser/cdproject/public_data/HC", 0)

#f = psd를 측정한 frequency의 1d ndarray
#x = frequency x channel의 psd(2d ndarray)를 요소로 갖는 List
f, x_ADHD, y_ADHD, s_ADHD = preprocess_data(x_ADHD, y_ADHD, s_ADHD)
f, x_HC, y_HC, s_HC = preprocess_data(x_HC, y_HC, s_HC)

#x = frequecy x channel의 psd(2d ndarray)를 요소로 갖는 List(ADHD + HC)
x: np.ndarray = np.concatenate((x_ADHD, x_HC), axis=0) 
y: np.ndarray = np.concatenate((y_ADHD, y_HC), axis=0) 
s: np.ndarray = np.concatenate((s_ADHD, s_HC), axis=0)

cnn = cnn((len(f), len(x[0][0]), 1))
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', metrics.Precision(), metrics.Recall(), metrics.AUC()])

sgkf = StratifiedGroupKFold(n_splits=10)
i = 1
for (train_index, test_index) in sgkf.split(x, y, s):
    x_train = x[train_index]
    y_train = y[train_index]
    s_train = s[train_index]

    x_test = x[test_index]
    y_test = y[test_index]
    s_test = s[test_index]

    print(f"fold {i}")
    history = cnn.fit(x_train, y_train, epochs=100, batch_size=5)
    for key in history.history.keys():
        print(f"{key}: {history.history[key]}")
    metric = cnn.evaluate(x_test, y_test)
    print(metric)
    print("\n")
    i += 1

/home/unixuser/.local/lib/python3.10/site-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(
/home/unixuser/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1728998792.726604    3830 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728998793.059503    3830 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728998793.059752    3830 cuda_executor.cc:1001] could 

NameError: name 'metrics' is not defined